Q1


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/ham_spam.csv', encoding='latin-1')
data = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1, inplace = True)
df.rename(columns = {"v2":"text", "v1":"output"}, inplace = True)
df.head(2)

In [ ]:
import nltk
from nltk import sent_tokenize,word_tokenize
import re
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
stopword = stopwords.words('english')

In [ ]:
for i in range(len(df)):
  text = df.iloc[i][1]
  text = word_tokenize(text)
  text.append(" playing")
  new_text = [ps.stem(word.lower()) for word in text if word.lower() not in stopword and word.isalpha()]
  new_text = ' '.join(new_text)
  df.iloc[i][1] = new_text

In [ ]:
df.head(2)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1000)

In [ ]:
X = cv.fit_transform(df['text']).toarray()
X.shape

In [ ]:
tdm = pd.DataFrame(X, columns = cv.get_feature_names_out())
tdm.head(3)

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
y = LE.fit_transform(df['output'])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tdm, y, train_size = 0.7, random_state = 43)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
cm

Q2

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
x = iris.data
y = iris.target

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.7, random_state = 43)

In [ ]:
import numpy as np
size = x_train.shape[0]
label_prob = {}
for outcome in np.unique(y_train):
  outcome_count = sum(outcome==y_train)
  label_prob[outcome] = outcome_count/size
print(label_prob)

In [ ]:
target_names = list(iris.target_names)
target_names

In [ ]:
df = pd.DataFrame(x_train,columns = iris.feature_names)
df['target'] = y_train
df.head(5)

In [ ]:
def GaussianProb(mean,var,new_x):
  exp = np.exp((-(new_x - mean)**2)/(2*var))
  start = (1/(np.sqrt(2*np.pi*var))) 
  return start*exp

In [ ]:
final_table = []
for i in range(3):
  new_table = []
  temp_df = df[df['target'] == i]
  for feature in iris.feature_names:
    mean,var = temp_df[feature].mean() , temp_df[feature].var()
    new_table.append([mean,var])
  final_table.append(new_table)

In [ ]:
final_table[0]

In [ ]:
test = x_test[0]
test

In [ ]:
x_train[:5]

In [ ]:
y_pred = []
for i in range(len(x_test)): 
  test = x_test[i]
  probs = {}
  count = 0
  for each_label in final_table:  
    prob = label_prob[count]  
    feature_count = 0 
    for stats in each_label:  
     prob = prob * GaussianProb(stats[0],stats[1],test[feature_count])
     feature_count+=1 
    count+=1  
    probs[count-1] = prob 
  output =max(probs, key=probs.get)
  y_pred.append(output)

In [ ]:
for i in range(len(y_test)):
  print(y_pred[i], y_test[i])

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(y_test, y_pred))

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(y_test, y_pred))

Q3

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
knn = KNeighborsClassifier()
params = {
    'n_neighbors' : [1,2,3,4,5],
     'weights' : ['uniform', 'distance'],
     'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
     'p' : [1,2]
}

In [ ]:
classifier = GridSearchCV(estimator = knn, param_grid = params)
classifier.fit(iris.data,iris.target)

In [ ]:
print(classifier.best_params_)
print(classifier.best_score_)